In [1]:
import numpy as np
import time 
import matplotlib.pyplot as plt
from util import *

%load_ext autoreload
%autoreload 2

In [2]:
MicroArray,Labels = read_data(mode = "CellType",th = 20)
MicroArray = pca(MicroArray,ratio=0.95)
print MicroArray.shape,Labels.shape

class_num = len(np.unique(Labels))
input_dim = MicroArray.shape[1]
print class_num,input_dim

Loading Previous Data!
883
(1077, 883) (1077, 1)
19 883


In [3]:
# np.save("./res/ds_pca.npy",MicroArray)
# np.save("./res/ds_labels.npy",Labels)

In [4]:
## shuffle data and divide it into 5 parts
from util import *

MicroLabels = Labels
MicroArray_train,MicroLabels_train,MicroArray_test,MicroLabels_test = division(MicroArray,MicroLabels)

print MicroArray_train.shape
print MicroArray_test.shape
print MicroLabels_train.shape
print MicroLabels_test.shape

(869, 883)
(208, 883)
(869, 1)
(208, 1)


# use tensorflow 

In [5]:
import tensorflow as tf

def save_model(name):
    saver = tf.train.Saver()
    path = "./models/%s.ckpt" % name
    save_path = saver.save(sess,path)
    print("Model saved in file: %s" % save_path)
    
def load_model(name):
    saver = tf.train.Saver()
    path = "./models/%s.ckpt" % name
    saver.restore(sess, path)
    print("Model restored.")

In [6]:
# creating nodes for the input images and target output classes.
sess = tf.InteractiveSession()

#placeholders
x = tf.placeholder(tf.float32, shape=[None,input_dim])
y_ = tf.placeholder(tf.float32,shape=[None,1])


keep_prob = tf.placeholder(tf.float32,shape=[])

reg_penalty = tf.placeholder(tf.float32,shape=[])

starter_learning_rate = tf.placeholder(tf.float32,shape=[])

In [7]:
global_step_tensor = tf.Variable(0, trainable=False, name='global_step')
W1 = tf.Variable(
    tf.random_normal(shape = [input_dim,2000],mean=0.01,stddev=0.1),name = "W1"
)
b1 = tf.Variable(
    tf.random_normal(shape = [2000],mean=0.01,stddev=0.1),name = "b1"
)
n1 = reg_penalty * (tf.reduce_sum(tf.square(W1)) + tf.reduce_sum(tf.square(b1)))

y1 = tf.matmul(x,W1) + b1
o1 = tf.nn.relu(y1)
d1 = tf.nn.dropout(o1,keep_prob=keep_prob)


W2 = tf.Variable(
    tf.random_normal(shape = [2000,1000],mean=0.01,stddev=0.1),name = "W2"
)
b2 = tf.Variable(
    tf.random_normal(shape = [1000],mean=0.01,stddev=0.1),name = "b2"
)
n2 = reg_penalty * (tf.reduce_sum(tf.square(W2)) + tf.reduce_sum(tf.square(b2)))

y2 = tf.matmul(d1,W2) + b2
o2 = tf.nn.relu(y2)
d2 = tf.nn.dropout(o2,keep_prob=keep_prob)

W3 = tf.Variable(
    tf.random_normal(shape = [1000,500],mean=0.01,stddev=0.1),name = "W3"
)
b3 = tf.Variable(
    tf.random_normal(shape = [500],mean=0.01,stddev=0.1),name = "b3"
)
n3 = reg_penalty * (tf.reduce_sum(tf.square(W3)) + tf.reduce_sum(tf.square(b3)))

y3 = tf.matmul(d2,W3) + b3
o3 = tf.nn.relu(y3)
d3 = tf.nn.dropout(o3,keep_prob=keep_prob)


W4 = tf.Variable(
    tf.random_normal(shape = [500,class_num],mean=0.01,stddev=0.1),name = "W4"
)
b4 = tf.Variable(
    tf.random_normal(shape = [class_num],mean=0.01,stddev=0.1),name = "b4"
)
n4 = reg_penalty * (tf.reduce_sum(tf.square(W4)) + tf.reduce_sum(tf.square(b4)))

y4 = tf.matmul(d3,W4) + b4


onehot_labels = tf.one_hot(indices=tf.cast(y_, tf.int32), depth = class_num)

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=onehot_labels, logits=y4))

#loss = cross_entropy
loss = cross_entropy + n1 + n2 + n3 + n4

In [8]:
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step_tensor,
                                       10000, 0.96, staircase=True)

opt = tf.train.AdamOptimizer(
    learning_rate = learning_rate,
    beta1 = 0.9,
    beta2 = 0.999,
    epsilon = 1e-08,
    name='Adam'
)

train_step = opt.minimize(loss,
    global_step = global_step_tensor)

In [9]:
#
tf.global_variables_initializer().run()

In [10]:
def eval_accuracy(data,labels):
    prediction = tf.identity(y4).eval(feed_dict={x: data,keep_prob:1})
    prediction = np.argmax(prediction,axis=1).reshape([-1,1])
    return np.mean(prediction == labels)

def predict(data):
    prediction = tf.identity(y4).eval(feed_dict={x: data,keep_prob:1})
    prediction = np.argmax(prediction,axis=1)
    return prediction
    
def eval_mse_loss(data,labels,reg):
    entropy = tf.identity(cross_entropy).eval(feed_dict={x: data, y_: labels,reg_penalty:reg,keep_prob:1})
    return entropy

def eval_loss(data,labels,reg):
    l = tf.identity(loss).eval(feed_dict={x: data, y_: labels,reg_penalty:reg,keep_prob:1})
    return l

def valid_batch(batch,th=0.2):
    uni = np.unique(batch)
    for i in uni:
        if(np.sum(batch==i) > th*batch.shape[0]):
            return False
    return True

In [11]:
train_loss = []
test_loss = []
train_acc = []
test_acc = []

In [13]:
batch_size = 200
epoch = int( MicroArray_train.shape[0] / batch_size)
print epoch
epoch_iter = 500
max_test = 0

rp = 1e-3
kp = 0.6
sl = 1e-3

modelname = "ct_lr_%f_l2_%f+dp_%f" % (sl,rp,kp)
print modelname

for step in range(epoch_iter * epoch):
  random = np.random.choice(MicroArray_train.shape[0],batch_size,replace=False)
  while(not valid_batch(Labels[random])):
        random = np.random.choice(MicroArray_train.shape[0],batch_size,replace=False)
      
  feed_dict = {
      x: MicroArray_train[random,:], 
      y_: MicroLabels_train[random],
      reg_penalty:rp,
      keep_prob:kp,
      starter_learning_rate:sl
    }

  _,a = sess.run([train_step,loss],feed_dict=feed_dict)

  if(step % epoch == 0):
    train_loss.append(a)
    b = eval_loss(MicroArray_test,MicroLabels_test,rp)
    test_loss.append(b)

    train_accuracy = eval_accuracy(MicroArray_train,MicroLabels_train)
    test_accuracy = eval_accuracy(MicroArray_test,MicroLabels_test)
    train_acc.append(train_accuracy)
    test_acc.append(test_accuracy)
    
    if(test_accuracy > max_test):
        max_test = test_accuracy
        save_model(modelname)
    print "epoch:%d   train loss:%f ,test loss :%f,train accuracy:%f,test accuracy:%f" % \
        (step/epoch,a,b,train_accuracy,test_accuracy)

4
ct_lr_0.001000_l2_0.001000+dp_0.600000
Model saved in file: ./models/ct_lr_0.001000_l2_0.001000+dp_0.600000.ckpt
epoch:0   train loss:2353.953369 ,test loss :1464.682617,train accuracy:0.657077,test accuracy:0.600962


KeyboardInterrupt: 

In [ ]:
load_model(modelname)
print eval_accuracy(MicroArray_test,MicroLabels_test)

In [ ]:
plt.plot(train_loss,"b")
plt.plot(test_loss,"g")
plt.show()

In [ ]:
plt.plot(train_acc,"b")
plt.plot(test_acc,"g")
plt.show()